# Подзапросы

Подзапросы могут применяться в следующих частях основного запроса:

в операторе FROM;\
в операторе SELECT (если запрос возвращает один столбец с одним значением);\
в операторах WHERE и HAVING (если запрос возвращает один столбец с одним или несколькими значениями);\
в операторе CASE при формировании продвинутых условных конструкций.

SELECT column_1\
FROM (\
    SELECT column_1, column_2\
    FROM table\
) AS subquery_1

По сути подзапрос — это такая же таблица, только временная. Она формируется в процессе выполнения основного запроса и нигде не сохраняется.

В примере выше сначала будет выполнен подзапрос, который отберёт колонки column_1 и column_2 из таблицы table, а затем уже из образовавшейся таблицы основной запрос выберет колонку column_1.

Важный момент: при использовании подзапроса в блоке FROM сформированной в подзапросе таблице необходимо присвоить какой-нибудь алиас, иначе основной запрос не сработает. В примере выше мы обозначили результат подзапроса как subquery_1.

К колонкам из подзапроса можно применять агрегирующие функции — так же, как если бы мы обращались к колонкам исходных таблиц:

SELECT MAX(column_sum) AS max_sum\
FROM (\
    SELECT column_1, SUM(column_2) AS column_sum\
    FROM table\
    GROUP BY column_1\
) AS subquery_1

Здесь сначала в подзапросе мы сгруппируем данные по колонке column_1, посчитав для каждой группы сумму значений в колонке column_2, а затем уже в основном запросе найдём максимальное значение среди всех сумм.

# WITH и Табличные выражения

Табличные выражения — это временные таблицы, существующие только для одного запроса. Их основное предназначение заключается в разбиении сложных запросов на несколько частей.

Табличные выражения создаются так:

WITH \
subquery_1 AS (\
    SELECT column_1, column_2\
    FROM table\
)

SELECT column_1\
FROM subquery_1

Оператор WITH может содержать несколько табличных выражений, причём к указанным ранее выражениям можно обращаться в последующих выражениях:

WITH \
subquery_1 AS (\
    SELECT column_1, column_2, column_3\
    FROM table\
),\
subquery_2 AS (\
    SELECT column_1, column_2\
    FROM subquery_1\
)

SELECT column_1\
FROM subquery_2


# WHERE и подзапросы

В первую очередь важно понять, что подзапрос, возвращающий одно значение, может использоваться совместно с операторами сравнения.

Иными словами, вы можете составить подзапрос, возвращающий одно значение (как правило, это результат работы агрегирующих функций), а затем сравнить с этим значением (т.е. с подзапросом) все значения в одной из колонок таблицы.

Давайте представим, что нам нужно сравнить значения в каком-то столбце с максимальным, минимальным или средним значением в этом же столбце — такая задача довольно часто встречается на практике. Сделать это в рамках одного запроса точно не получится, поскольку, как мы уже знаем, агрегирующие функции нельзя применять в блоке WHERE.

SELECT column\
FROM table\
WHERE column = (SELECT MAX(column) FROM table) 

# Дополнительные функции

Результат подзапроса, возвращающего столбец с одним значением, также можно использовать в арифметических операциях:

SELECT column\
FROM table\
WHERE column = (SELECT MAX(column) FROM table) - 100

Более того, с помощью подзапросов вы можете брать необходимые вам значения из нескольких разных таблиц и использовать их в качестве переменных внутри основного запроса:

SELECT column\
FROM table\
WHERE column >= (SELECT MAX(column_1) FROM table_1) - 100\
    AND column <= (SELECT MAX(column_2) FROM table_2)

То есть в данном случае результат выполнения подзапроса можно рассматривать как некоторую переменную. Со временем число записей в таблицах может меняться (допустим, появляются новые товары по новым ценам или новые заказы с более поздними датами), и наши подзапросы будут это учитывать.

* Если же в одном запросе используется несколько разных «переменных» из подзапросов или к одному и тому же подзапросу нужно обращаться несколько раз, тогда имеет смысл вынести эти подзапросы в начало основного запроса в виде табличных выражений в блоке WITH.

WITH \
subquery AS (\
    SELECT MAX(column_2)\
    FROM table_2\
)

SELECT column_1\
FROM table_1\
WHERE column_1 = (SELECT * FROM subquery) 

Обратите внимание на запись со «звёздочкой». Дело в том, что обратиться к этим «переменным» просто по имени табличного выражения не получится — придётся отдельным подзапросом из табличного выражения выбрать рассчитанное значение. Самый простой вариант — написать подзапрос с SELECT * из табличного выражения.

Если «переменных» несколько, то запрос может выглядеть так:

WITH\
subquery_1 AS (\
    SELECT MAX(column_1)\
    FROM table_1\
),\
subquery_2 AS (\
    SELECT MAX(column_2)\
    FROM table_2\
)


SELECT column\
FROM table\
WHERE column >= (SELECT * FROM subquery_1) - 100\
    AND column <= (SELECT * FROM subquery_2)